In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [35]:
df = pd.read_csv(r"C:\Users\hamir\Desktop\Wind Turbine\data\df.csv")
df.head()

,power_kw,wind_speed_ms,theoretical_power_kw,wind_direction_deg,year,month,day,hour
0,380.047791,5.311336,416.328908,259.994904,2018,1,1,0
1,453.769196,5.672167,519.917511,268.641113,2018,1,1,0
2,306.376587,5.216037,390.900016,272.564789,2018,1,1,0
3,419.645905,5.659674,516.127569,271.258087,2018,1,1,0
4,380.650696,5.577941,491.702972,265.674286,2018,1,1,0


In [16]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

In [36]:
#features = [ 'power_kw', 'wind_speed_ms', 'theoretical_power_kw', 'wind_direction_deg', 'month', 'hour']

features = ['power_kw', 'wind_speed_ms', 'theoretical_power_kw', 'wind_direction_deg', 'month', 'day', 'hour']

training_data = df[features]
X = training_data.drop(columns=['power_kw'], axis = 1).values
Y= training_data['power_kw'].values

In [37]:
x_train , x_test , y_train , y_test = train_test_split(X, Y, test_size=0.20, random_state=1234)

print("Dimensions of train set:")
print(f"X_train: {x_train.shape}, Y_train: {y_train.shape}")
print("Dimensions of test set:")
print(f"X_test: {x_test.shape}, Y_test: {y_test.shape}")


Dimensions of train set:
X_train: (40424, 6), Y_train: (40424,)
Dimensions of test set:
X_test: (10106, 6), Y_test: (10106,)


In [6]:
import multiprocessing
n_cpus = multiprocessing.cpu_count()
n_cpus

4

In [7]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.feature_selection import SelectFromModel
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV


# # Hyperparameter Tuning
# rf = RandomForestRegressor()
# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_features': ['sqrt', 'log2'],
#     'max_depth': [None, 10, 20, 30, 40, 50],
#     'min_samples_split': [2, 4, 6, 8, 10],
#     'min_samples_leaf': [1, 2, 4, 6, 8]
# }
# rf_cv = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=n_cpus )
# rf_cv.fit(x_train, y_train)
# best_params = rf_cv.best_params_  

# print(best_params)


In [9]:
def train_model( models, x_train, x_test, y_train, y_test):
    
    model = models
    model.fit(x_train,y_train)
    y_pred_test = model.predict(x_test)
    y_pred_train = model.predict(x_train)
    y_total_prediction = model.predict(X)
    
    print("\t\tError Table")
    print('Mean Absolute Error      : ', metrics.mean_absolute_error(y_test, y_pred_test))
    print('Mean Squared  Error      : ', metrics.mean_squared_error(y_test, y_pred_test))
    print('Root Mean Squared  Error : ', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))
    print('Accuracy on Training set   : ', metrics.r2_score(y_train,y_pred_train))
    print('Accuracy on Testing set  : ', metrics.r2_score(y_test,y_pred_test))
    return y_total_prediction, model

In [13]:
def visualizing_power(y_total_prediction, y_actual):
    y_real = y_actual
    y_prediction = y_total_prediction
    TP = df['theoretical_power_kw']
    number = len(df['wind_speed_ms'])
    aa = [x for x in df['wind_speed_ms']]
    
    plt.figure(figsize=(21, 6))
    
    sns.scatterplot(x=aa, y=y_real[:number], color='cornflowerblue', label='Real Power Production', marker='o')
    sns.scatterplot(x=aa, y=y_prediction[:number], color='#3CB371', label='Predicted Power Production', alpha=0.3, marker='P')
    sns.lineplot(x='wind_speed_ms', y='theoretical_power_kw', color='crimson', label='Theoretical Power Production', data=df)
    
    plt.xlabel('Wind Speed (m/s)', size=10)
    plt.ylabel('Power Production (kW)', size=10)
    plt.title('Wind Turbine Power Production Prediction', size=20)
    plt.legend(fontsize=10)
    plt.show()


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

# Hyperparameter Tuning
rf = RandomForestRegressor()
param_dist = {
    'n_estimators': sp_randint(100, 500),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': sp_randint(2, 10),
    'min_samples_leaf': sp_randint(1, 8)
}
n_iter_search = 50
rf_cv = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=n_iter_search, cv=5, n_jobs=n_cpus)
rf_cv.fit(x_train, y_train)
best_params = rf_cv.best_params_  

print(best_params)


{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 112}


In [19]:
param_best = {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 112}

In [38]:
y_predicted, RFOREST = train_model( RandomForestRegressor(**best_params),  x_train , x_test , y_train , y_test)
#visualizing_power(y_predicted, Y)

		Error Table
Mean Absolute Error      :  72.77467387308954
Mean Squared  Error      :  31235.62040054334
Root Mean Squared  Error :  176.73601896767772
Accuracy on Training set   :  0.9967842249597506
Accuracy on Testing set  :  0.982062248277174


In [45]:
import pickle

# Train and get the best random forest model

# Save the trained model to a file
with open('best_rf.pickle', 'wb') as f:
    pickle.dump(RFOREST, f)
